# 11-1 데이터 집계(1/11/2019)(218p)

수집한 데이터를 바탕으로 평균이나 합 등을 구하여 의미 있는 값을 도출해 내는 것을 '집계'라고 한다.

groupby 메서드로 평균값을 구하는 과정을 통해 데이터 집계가 무엇인지 알아보자

In [2]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep='\t')

In [5]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [4]:
avg_lifeExp_by_year = df.groupby('year').lifeExp.mean()
avg_lifeExp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

## groupby 메서드는 어떤 과정을 통해 데이터를 집계할까?

### 분할 (어떤 기준으로 데이터를 나누기)

In [6]:
years = df.year.unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

### 반영 (함수 등을 적용하여 데이터를 처리하기)

In [9]:
y1952 = df.loc[df.year == 1952]
y1952.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
12,Albania,Europe,1952,55.230,1282697,1601.056136
24,Algeria,Africa,1952,43.077,9279525,2449.008185
36,Angola,Africa,1952,30.015,4232095,3520.610273
48,Argentina,Americas,1952,62.485,17876956,5911.315053


In [10]:
y1952_mean = y1952.lifeExp.mean()
y1952_mean

49.05761971830987

과정 9, 10을 반복하여 나머지 연도들의 평균값을 구하면 '반영' 작업이 끝난다.

In [11]:
y1957 = df.loc[df.year==1957]
y1957.head()

,country,continent,year,lifeExp,pop,gdpPercap
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
13,Albania,Europe,1957,59.280,1476505,1942.284244
25,Algeria,Africa,1957,45.685,10270856,3013.976023
37,Angola,Africa,1957,31.999,4561361,3827.940465
49,Argentina,Americas,1957,64.399,19610538,6856.856212


In [12]:
y1957_mean = y1957.lifeExp.mean()
y1957_mean

51.50740112676054

In [14]:
y1962 = df.loc[df.year==1962]
y1962.head()

,country,continent,year,lifeExp,pop,gdpPercap
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
14,Albania,Europe,1962,64.820,1728137,2312.888958
26,Algeria,Africa,1962,48.303,11000948,2550.816880
38,Angola,Africa,1962,34.000,4826015,4269.276742
50,Argentina,Americas,1962,65.142,21283783,7133.166023


In [15]:
y1962_mean = y1962.lifeExp.mean()
y1962_mean

53.60924901408449

In [17]:
y1967 = df.loc[df.year == 1967]
y1967.head()

,country,continent,year,lifeExp,pop,gdpPercap
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
15,Albania,Europe,1967,66.220,1984060,2760.196931
27,Algeria,Africa,1967,51.407,12760499,3246.991771
39,Angola,Africa,1967,35.985,5247469,5522.776375
51,Argentina,Americas,1967,65.634,22934225,8052.953021


In [18]:
y1967_mean = y1967.lifeExp.mean()
y1967_mean

55.67828957746479

### 결합 (처리한 결과를 다시 합치기)

In [19]:
df2 = pd.DataFrame({'year':[1952, 1957, 1962, 1967],
                   "":[y1952_mean, y1957_mean, y1962_mean, y1967_mean]})
df2

,,year
0,49.057620,1952
1,51.507401,1957
2,53.609249,1962
3,55.678290,1967


## agg 메서드로 사용자 함수와 groupby 메서드 조합하기

### 평균값을 구하는 사용자 함수와 groupby 메서드 조합하기

In [20]:
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
        
    return sum / n

In [21]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
agg_my_mean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

### 2개의 인자값을 받아 처리하는 사용자 함수와 groupby 메서드 조합하기

#### 연도별 평균 수명에서 전체 평균 수명을 뺀 값을 구하기

In [22]:
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [23]:
global_mean = df.lifeExp.mean()
global_mean

59.47443936619713

In [24]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)
agg_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

## 여러개의 집계 메서드 한 번에 사용하기

### 리스트에 담아서 전달하기

In [27]:
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142.0,49.057620,12.225956
1957,142.0,51.507401,12.231286
1962,142.0,53.609249,12.097245
1967,142.0,55.678290,11.718858
1972,142.0,57.647386,11.381953
1977,142.0,59.570157,11.227229
1982,142.0,61.533197,10.770618
1987,142.0,63.212613,10.556285
1992,142.0,64.160338,11.227380


### 딕셔너리에 담아서 전달하기 (키: 집계 메서드를 적용할 열 이름, 값: 집계 메서드)

In [29]:
gdf_dict = df.groupby('year').agg({'lifeExp':'mean', 'pop':'median', 'gdpPercap':'median'})
gdf_dict

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334741
1972,57.647386,5877996.5,3339.129407
1977,59.570157,6404036.5,3798.609244
1982,61.533197,7007320.0,4216.228428
1987,63.212613,7774861.5,4280.300366
1992,64.160338,8688686.5,4386.085502


# 11-2 데이터 변환

## 표준점수 계산하기

In [30]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

#### transform 메서드 사용

In [32]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
transform_z

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
5      -1.882224
6      -2.012809
7      -2.121069
8      -2.002813
9      -2.011488
10     -1.919077
11     -1.919936
12      0.504859
13      0.635469
14      0.926719
15      0.899551
16      0.882328
17      0.833673
18      0.825097
19      0.832432
20      0.660943
21      0.686480
22      0.810767
23      0.779886
24     -0.489174
25     -0.476025
26     -0.438633
27     -0.364480
28     -0.274943
29     -0.138606
          ...   
1674   -1.153155
1675   -0.974833
1676   -0.762541
1677   -0.605105
1678   -0.438681
1679   -0.356947
1680   -0.574157
1681   -0.607491
1682   -0.627106
1683   -0.675005
1684   -0.662486
1685   -0.728956
1686   -0.901731
1687   -1.173861
1688   -1.608598
1689   -2.143415
1690   -2.158168
1691   -2.039541
1692   -0.049617
1693   -0.084897
1694   -0.103433
1695   -0.143639
1696   -0.176805
1697   -0.168889
1698   -0.108647
1699   -0.081621
1700   -0.336974
1701   -1.5749

In [36]:
df.shape

(1704, 6)

In [37]:
transform_z.shape

(1704,)

### 누락값을 평균값으로 처리하기

In [38]:
import seaborn as sns
import numpy as np

np.random.seed(42)

In [39]:
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NAN
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


### 빈 값을 단순히 total_bill의 평균값으로 채우면 안됌.(남자가 여자보다 많기 때문!)

In [46]:
tips_10.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


In [47]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [48]:
# 이해안됌
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
24,19.82,3.18,Male,No,Sat,Dinner,2,19.8200
6,8.77,2.00,Male,No,Sun,Dinner,2,8.7700
153,NaN,2.00,Male,No,Sun,Dinner,4,17.9525
211,NaN,5.16,Male,Yes,Sat,Dinner,4,17.9525
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.9300
176,NaN,2.00,Male,Yes,Sun,Dinner,2,17.9525
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.4400
124,12.48,2.52,Female,No,Thur,Lunch,2,12.4800
9,14.78,3.23,Male,No,Sun,Dinner,2,14.7800
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.3800
